# Вебинар 6. Двухуровневые модели рекомендаций


### Зачем 2 уровня?
- Классические модели классификации (lightgbm) зачастую работают лучше, чем рекоммендательные модели (als, lightfm)
- Данных много, предсказаний много (# items * # users) --> с таким объемом lightgbm не справляется
- Но рекомендательные модели справляются!

Отбираем top-N (200) *кандидатов* с помощью простой модели (als) --> переранжируем их сложной моделью (lightgbm)
и выберем top-k (10).

---

### Как отбирать кандидатов?

Вариантов множество. Тут нам поможет *MainRecommender*. Пока в нем реализованы далеко не все возможные способы генерации кандидатов

- Генерируем топ-k кандидатов
- Качество кандидатов измеряем через **recall@k**
- recall@k показывает какую долю из купленных товаров мы смогли выявить (рекомендовать) нашей моделью

----

# Практическая часть

# Import libs

In [67]:
# !pip install lightfm
!pip install implicit==0.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  PATH = '/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_3/'

else:

  PATH = 'D:/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)


pd.options.mode.chained_assignment = None
from tqdm import tqdm    

# # Написанные нами функции
# from metrics import precision_at_k, recall_at_k
# from utils import prefilter_items
# from recommenders import MainRecommender


try:

  from metrics import precision_at_k, recall_at_k
  from utils import prefilter_items
  from recommenders import MainRecommender

except:
  
  sys.path.insert(0,'/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_6/webinar_6')
  from metrics import precision_at_k, recall_at_k
  from utils import prefilter_items
  from recommenders import MainRecommender


## Read data

In [70]:
# data = pd.read_csv('../data/retail_train.csv')
# item_features = pd.read_csv('../data/product.csv')
# user_features = pd.read_csv('../data/hh_demographic.csv')


try:
  
  data = pd.read_csv('../data/retail_train.csv')
  item_features = pd.read_csv('../data/product.csv')
  user_features = pd.read_csv('../data/hh_demographic.csv')

except:

  data = pd.read_csv(PATH + 'data/retail_train.csv')
  item_features = pd.read_csv(PATH + '/data/product.csv') 
  user_features = pd.read_csv(PATH + '/data/hh_demographic.csv') 

# Process features dataset

In [71]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'

In [72]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

In [73]:
# import implicit

In [74]:
# implicit.gpu.HAS_CUDA

# Split dataset for train, eval, test

In [75]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)


VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [76]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [77]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [78]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [79]:
# выше видим разброс по пользователям и товарам

In [80]:
data_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


# Prefilter items

In [81]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


# Make cold-start to warm-start

In [82]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (861404, 13) Users: 2495 Items: 5001
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


In [83]:
# Теперь warm-start по пользователям

# Init/train recommender

In [84]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

### Варианты, как получить кандидатов

Можно потом все эти варианты соединить в один

(!) Если модель рекомендует < N товаров, то рекомендации дополняются топ-популярными товарами до N

In [85]:
# Берем тестового юзера 2375

In [86]:
recommender.get_als_recommendations(2375, N=5)

[1106523, 899624, 871756, 925862, 1116376]

In [87]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [88]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 1042907, 1115576, 15778319]

In [89]:
# recommender.get_similar_users_recommendation(2375, N=5)

# Eval recall of matching

### Измеряем recall@k

Это будет в ДЗ: 

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [90]:
ACTUAL_COL = 'actual'

In [91]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [92]:
# N = Neighbors
N_PREDICT = 100 

In [93]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
# result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=50))
# result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=50))

CPU times: user 6.37 s, sys: 27.7 ms, total: 6.4 s
Wall time: 6.4 s


In [94]:
%%time
# result_eval_matcher['sim_user_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 5.48 µs


### Пример оборачивания

In [95]:
# # сырой и простой пример как можно обернуть в функцию
def evalRecall(df_result, target_col_name, recommend_model):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=25))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=N_PREDICT), axis=1).mean()

In [96]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [97]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

### Recall@50 of matching

In [98]:
TOPK_RECALL = 50

In [99]:
sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

[('own_rec', 0.06525657038145165)]

### Precision@5 of matching

In [100]:
TOPK_PRECISION = 5

In [101]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec', 0.17712691771269176)]

# Ranking part

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_ranking
- Обучаем *только* на выбранных кандидатах
- Я *для примера* сгенерирую топ-50 кадидиатов через get_own_recommendations
- (!) Если юзер купил < 50 товаров, то get_own_recommendations дополнит рекоммендации топ-популярными

In [102]:
# 3 временных интервала
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

## Подготовка данных для трейна

In [103]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [104]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))


In [105]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."


In [106]:
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [107]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [108]:
df_match_candidates.head(4)

,user_id,item_id
0,2070,1105426
0,2070,1097350
0,2070,879194
0,2070,948640


### Check warm start

In [109]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (215100, 2) Users: 2151 Items: 4585


### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [110]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

In [111]:
df_ranker_train.head()

,user_id,item_id,target
2104867,2070,1019940,1
2107468,2021,840361,1
2107469,2021,856060,1
2107470,2021,869344,1
2107471,2021,896862,1


#### Не хватает нулей в датасете, поэтому добавляем наших кандитатов в качество нулей

In [112]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [113]:
df_ranker_train.target.value_counts()

0.0    200105
1.0     11886
Name: target, dtype: int64

In [114]:
df_ranker_train.head(2)

,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0


(!) На каждого юзера 50 item_id-кандидатов

In [115]:
df_ranker_train['target'].mean()

0.05606841799887731

![hard_choice.png](attachment:hard_choice.png)

Слайд из [презентации](https://github.com/aprotopopov/retailhero_recommender/blob/master/slides/retailhero_recommender.pdf) решения 2-ого места X5 Retail Hero

- Пока для простоты обучения выберем LightGBM c loss = binary. Это классическая бинарная классификация
- Это пример *без* генерации фич

## Подготавливаем фичи для обучения модели

**Фичи user_id:**
    - Средний чек
    - Средняя сумма покупки 1 товара в каждой категории
    - Кол-во покупок в каждой категории
    - Частотность покупок раз/месяц
    - Долю покупок в выходные
    - Долю покупок утром/днем/вечером

**Фичи item_id**:
    - Кол-во покупок в неделю
    - Среднее кол-во покупок 1 товара в категории в неделю
    - (Кол-во покупок в неделю) / (Среднее ол-во покупок 1 товара в категории в неделю)
    - Цена (Можно посчитать из retil_train.csv)
    - Цена / Средняя цена товара в категории
    
**Фичи пары user_id - item_id**
    - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
    - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
    - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

In [116]:
display(df_ranker_train.shape)
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')
display(df_ranker_train.shape)
df_ranker_train.head(2)

(211991, 3)

(211991, 16)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [117]:
# data_train_ranker_columns = data_train_ranker.columns
# df_ranker_train_columns = df_ranker_train.columns
data_train_ranker.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0


In [118]:
# data_train_ranker

# data_train_ranker = data_train_ranker[data_train_ranker_columns]
# df_ranker_train = df_ranker_train[df_ranker_train_columns]
df_ranker_train.head(2) 

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


Фичи user_id
<a id='section_id'></a>
<a name="id"></a>

In [119]:
# признаки клиента по скидкам
groups_by = ['retail_disc']

for group_by in groups_by:
  _ = data_train_ranker.groupby(USER_COL)[[group_by]].agg([np.max, np.min, np.size, np.mean, np.sum]).reset_index()
  _.columns = _.columns.get_level_values(level = 1) + _.columns.get_level_values(level = 0)
  _['number_retail_disc'] = _['amaxretail_disc'] - _['aminretail_disc']
  df_ranker_train = df_ranker_train.merge(_, on='user_id', how='left')

display(df_ranker_train.shape)
df_ranker_train[_.columns].head()

(211991, 22)

,user_id,amaxretail_disc,aminretail_disc,sizeretail_disc,meanretail_disc,sumretail_disc,number_retail_disc
0,2070,0.0,-22.07,204,-0.740049,-150.97,22.07
1,2070,0.0,-22.07,204,-0.740049,-150.97,22.07
2,2070,0.0,-22.07,204,-0.740049,-150.97,22.07
3,2070,0.0,-22.07,204,-0.740049,-150.97,22.07
4,2070,0.0,-22.07,204,-0.740049,-150.97,22.07


In [120]:
# признаки клиента по sales_value
groups_by = ['sales_value']
columns_users = []

for group_by in groups_by:
  _ = data_train_ranker.groupby(USER_COL)[[group_by]].agg([np.mean, np.sum, np.size]).reset_index()
  _.columns = _.columns.get_level_values(level = 1) + _.columns.get_level_values(level = 0)

  df_ranker_train = df_ranker_train.merge(_, on='user_id', how='left')
  # columns_users.append(__.columns.to_list)

display(df_ranker_train.shape)
df_ranker_train[_.columns].head()

(211991, 25)

,user_id,meansales_value,sumsales_value,sizesales_value
0,2070,3.025931,617.29,204
1,2070,3.025931,617.29,204
2,2070,3.025931,617.29,204
3,2070,3.025931,617.29,204
4,2070,3.025931,617.29,204


In [121]:
# признаки по неделям клиентов (длительность покупок, когда начал, последняя покупка)
groups_by = ['week_no']

for group_by in groups_by:
  _ = data_train_ranker.groupby(USER_COL)[[group_by]].agg([np.max, np.min, np.size]).reset_index()
  _.columns = _.columns.get_level_values(level = 1) + _.columns.get_level_values(level = 0)
  _['number_weeks'] = _['amaxweek_no'] - _['aminweek_no']
  df_ranker_train = df_ranker_train.merge(_, on='user_id', how='left')

display(df_ranker_train.shape)
df_ranker_train[_.columns].head()

(211991, 29)

,user_id,amaxweek_no,aminweek_no,sizeweek_no,number_weeks
0,2070,91,86,204,5
1,2070,91,86,204,5
2,2070,91,86,204,5
3,2070,91,86,204,5
4,2070,91,86,204,5


In [122]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


Фичи item_id

In [123]:
# Создадим датасет для генерации item_features
data_for_item_features = data_train_ranker.merge(item_features, on='item_id', how='inner')
data_for_item_features.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,2070,40618492260,594,1019940,1,1.0,311,-0.29,40,86,0.0,0.0,1232,GROCERY,National,SOFT DRINKS,SOFT DRINK BOTTLE NON-CARB (EX,20 OZ
1,2070,40630625006,594,1019940,1,1.0,311,-0.29,201,86,0.0,0.0,1232,GROCERY,National,SOFT DRINKS,SOFT DRINK BOTTLE NON-CARB (EX,20 OZ


In [124]:
df_ranker_train.head(2)
_.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,2070,40618492260,594,1019940,1,1.0,311,-0.29,40,86,0.0,0.0,1232,GROCERY,National,SOFT DRINKS,SOFT DRINK BOTTLE NON-CARB (EX,20 OZ
1,2070,40630625006,594,1019940,1,1.0,311,-0.29,201,86,0.0,0.0,1232,GROCERY,National,SOFT DRINKS,SOFT DRINK BOTTLE NON-CARB (EX,20 OZ


In [125]:
# признаки item по brand и department
groups_by = ['brand', 'department']

for group_by in groups_by:
  _ = data_for_item_features.groupby(groups_by)[['sales_value']].agg([np.size, np.mean, np.sum]).reset_index()
  _.columns = _.columns.get_level_values(level = 1) + _.columns.get_level_values(level = 0)
  df_ranker_train = df_ranker_train.merge(_, on=groups_by, how='left', suffixes = ('', '_'+ str(group_by)))

display(df_ranker_train.shape)

(211991, 35)

Как понимаю user-item признаки это что то уникальное в группировке user+item. У нас есть возможность сгурппировать item по department, brand, manufacturer итд и получить агрегированные значения (mean, sum, count)

In [126]:
groups_by = ['department', 'brand', 'manufacturer', 'sub_commodity_desc', 'curr_size_of_product']


for group_by in groups_by:
  _ = data_train_ranker.merge(item_features[['item_id', group_by]], on='item_id', how='inner')

  # гурппировка по department
  __ = _.groupby([USER_COL, group_by])\
                          [['sales_value']].agg([np.mean, np.sum, np.size]).reset_index()


  __.columns = __.columns.get_level_values(level = 1) + __.columns.get_level_values(level = 0) 

  df_ranker_train = df_ranker_train.merge(__, on=[USER_COL, group_by], how='left', suffixes = ('', '_'+ str(group_by)))
  df_ranker_train.head()

In [127]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [128]:
X_train.head(3)

,user_id,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,...,sizesales_value_brand,meansales_value_manufacturer,sumsales_value_manufacturer,sizesales_value_manufacturer,meansales_value_sub_commodity_desc,sumsales_value_sub_commodity_desc,sizesales_value_sub_commodity_desc,meansales_value_curr_size_of_product,sumsales_value_curr_size_of_product,sizesales_value_curr_size_of_product
0,2070,1105426,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,...,54.0,3.14717,166.8,53.0,NaN,NaN,NaN,7.824333,234.73,30.0
1,2070,1097350,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,...,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2070,879194,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,...,54.0,3.14717,166.8,53.0,7.99,7.99,1.0,NaN,NaN,NaN


In [129]:
# cat_feats = X_train.columns[2:].tolist()
# X_train[cat_feats] = X_train[cat_feats].astype('category')

# cat_feats

cat_feats = X_train.columns[:15].tolist()


for column in cat_feats:
    X_train[column].fillna(0, inplace=True)
    
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['user_id',
 'item_id',
 'manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [130]:
X_train.shape


(211991, 49)

## Обучение модели ранжирования

In [131]:
# делали на вебинаре, кусок с катбустом

# from catboost import CatBoost, Pool

# df_bin_feat = pd.get_dummies(X_train)

# model = CatBoost(params ={'objective': "Logloss",
#                           "iterations":1000,
#                           "max_depth":8,
#                           "task_type":"GPU",
#                           "devices":"0:1:2:3"
#             })
# model.fit(df_bin_juice, y_train, silent=True)

# train_preds = model.predict(df_bin_feat,prediction_type="Probability")

In [132]:
%%time
lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats,
                     n_jobs=-1,
#                      verbose=0
                   )

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


CPU times: user 38.4 s, sys: 87.5 ms, total: 38.5 s
Wall time: 20.4 s


In [133]:
df_ranker_predict = df_ranker_train.copy()

In [134]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [135]:
df_ranker_predict['proba_item_purchase'][:10]

0    0.000105
1    0.000012
2    0.000192
3    0.000013
4    0.000275
5    0.000035
6    0.000281
7    0.000014
8    0.000034
9    0.880500
Name: proba_item_purchase, dtype: float64

## Подведем итоги

    Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от own_recommendations, что является тренировочным сетом, и теперь наша задача предсказать и оценить именно на тестовом сете.

# Evaluation on test dataset

In [136]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Eval matching on test dataset

In [137]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 6.16 s, sys: 0 ns, total: 6.16 s
Wall time: 6.15 s


In [138]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

[('own_rec', 0.14441176470588235)]

## Eval re-ranked matched result on test dataset
    Вспомним df_match_candidates сет, который был получен own_recommendations на юзерах, набор пользователей мы фиксировали и он одинаков, значи и прогноз одинаков, поэтому мы можем использовать этот датафрейм для переранжирования.
    

In [139]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [140]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [141]:
print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.2042819843342037)
('own_rec', 0.14441176470588235)


/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_6/webinar_6/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


При использовании двухуровневой модели precision@5 вырос с 0,14 до 0,2

# Домашнее задание

**Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [142]:
result_eval_matcher = result_eval_matcher[['user_id', 'actual']]

models = {'own_rec': recommender.get_own_recommendations, 
          'sim_item_rec': recommender.get_similar_items_recommendation, 
          'als_rec': recommender.get_als_recommendations, 
          'sim_user_rec': recommender.get_similar_users_recommendation
          }


def evalRecall_my(df_result, target_col_name, recommend_model, lst_N):

    for N in tqdm(lst_N):
      for name, recommend_model in models.items():
        result_col_name = (f'result_{str(name)}_{str(N)}')
        df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=N))

    return (sorted(calc_recall(df_result, N), key=lambda x: x[1],reverse=True))
 
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()      


In [143]:
%%time

k_50 = evalRecall_my(result_eval_matcher, USER_COL, models, [50])

100%|██████████| 1/1 [06:57<00:00, 417.59s/it]


CPU times: user 7min 23s, sys: 4min 5s, total: 11min 28s
Wall time: 6min 58s


In [144]:
k_50

[('result_own_rec_50', 0.06525657038145165),
 ('result_als_rec_50', 0.0478519082774118),
 ('result_sim_item_rec_50', 0.03324361775791708),
 ('result_sim_user_rec_50', 0.006948819514303016)]

При k = 50 видим что лучшая вновь result_own 

In [145]:
%%time

own_rec_20_50_100_200_500 = evalRecall_my(result_eval_matcher, USER_COL, {'own_rec': recommender.get_own_recommendations}, [20, 50, 100, 200, 500])

100%|██████████| 5/5 [1:37:56<00:00, 1175.24s/it]


CPU times: user 1h 39min 2s, sys: 21min, total: 2h 2s
Wall time: 1h 38min


In [146]:
own_rec_20_50_100_200_500

[('result_own_rec_500', 0.18205324555508703),
 ('result_als_rec_500', 0.146353591484551),
 ('result_sim_item_rec_500', 0.136382765723656),
 ('result_own_rec_200', 0.13537278412833254),
 ('result_als_rec_200', 0.09822028233601446),
 ('result_own_rec_100', 0.09604492955885016),
 ('result_sim_item_rec_200', 0.08579617925573609),
 ('result_als_rec_100', 0.0697256361283774),
 ('result_own_rec_50', 0.06525657038145165),
 ('result_sim_item_rec_100', 0.05323383091193326),
 ('result_als_rec_50', 0.0478519082774118),
 ('result_own_rec_20', 0.039284276793729055),
 ('result_sim_item_rec_50', 0.03324361775791708),
 ('result_als_rec_20', 0.028865142910236535),
 ('result_sim_user_rec_500', 0.02477725443351606),
 ('result_sim_item_rec_20', 0.017980882882622493),
 ('result_sim_user_rec_200', 0.01575636644270816),
 ('result_sim_user_rec_100', 0.010828286476027462),
 ('result_sim_user_rec_50', 0.006948819514303016),
 ('result_sim_user_rec_20', 0.0036494539698160742)]

Понятно, что при большем k у нас будет выше метрика recall. Для данного датасета мы не видим что после какого то k идет затухание роста recall. Но сверху есть ограничние, потому что (с прицелом на использование двухуровневой модели), у нас стоит задача уменьшит размерность данных, поэтому оптимальным будет такой размер K при котором мы сможем провести расчет в разумное время и оперативно давать рекомендации клиенту в режиме онлайн. 
Без учета этого фактора оптимальным возможно будет комбинация recall и precision, что то типа f1.

**Задание 2.**

Обучите модель 2-ого уровня, при этом:

- Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар

- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_ranker

- Вырос ли precision@5 при использовании двухуровневой модели?

Выполнил задание 2 в середине скрипта (внутреннюю ссылку на него не смог сделать в колабе).

Оказалось что при использовании двухуровневой модели precision@5 вырос с 0,14 (own recommendtions) до 0,2
